In [1]:
!date "+[%F %R:%S] [INIT] HelloNSMC (using $CONDA_PREFIX)"
import time
t0 = time.time()

[2020-10-22 18:22:24] [INIT] HelloNSMC (using /home/chris/anaconda3/envs/trans2)


In [2]:
!pip install tqdm boto3 requests regex sentencepiece sacremoses urllib3 pandas pytorch-transformers

In [3]:
import os
if not os.path.exists("./nsmc/"):
    !git clone https://github.com/e9t/nsmc.git
else:
    print("Dataset already downloaded: nsmc")

Dataset already downloaded: nsmc


In [4]:
!date "+[%F %R:%S] [DONE] Environment & Dataset Installation"
print(f"td={time.time() - t0:.3f}")
t0 = time.time()

[2020-10-22 18:22:25] [DONE] Environment & Dataset Installation
td=1.019


In [5]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from pytorch_transformers import BertTokenizer, BertForSequenceClassification
from torch.optim import Adam
import torch.nn.functional as F

In [6]:
train_df = pd.read_csv('./nsmc/ratings_train.txt', sep='\t')
test_df = pd.read_csv('./nsmc/ratings_test.txt', sep='\t')
train_df.head(10)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
5,5403919,막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.,0
6,7797314,원작의 긴장감을 제대로 살려내지못했다.,0
7,9443947,별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단...,0
8,7156791,액션이 없는데도 재미 있는 몇안되는 영화,1
9,5912145,왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?,1


In [ ]:
train_df.dropna(inplace=True)
test_df.dropna(inplace=True)

train_df = train_df.sample(frac=0.4, random_state=999)
test_df = test_df.sample(frac=0.4, random_state=999)

In [ ]:
class NsmcDataset(Dataset):
    """ Naver Sentiment Movie Corpus Dataset """
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        text = self.df.iloc[idx, 1]
        label = self.df.iloc[idx, 2]
        return text, label

In [ ]:
nsmc_train_dataset = NsmcDataset(train_df)
train_loader = DataLoader(nsmc_train_dataset, batch_size=2, shuffle=True, num_workers=2)

In [ ]:
device = torch.device("cuda")
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased')
model.to(device)

In [ ]:
!date "+[%F %R:%S] [DONE] Pretrained Model Loading"
print(f"td={time.time() - t0:.3f}")
t0 = time.time()

In [ ]:
optimizer = Adam(model.parameters(), lr=1e-6)

epochs = 2
itr = 1
p_itr = 500
total_loss = 0
total_len = 0
total_correct = 0

model.train()
for epoch in range(epochs):

    for text, label in train_loader:
        optimizer.zero_grad()

        encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]
        padded_list =  [e + [0] * (512-len(e)) for e in encoded_list]
        sample = torch.tensor(padded_list)
        sample, label = sample.to(device), label.to(device)
        labels = torch.tensor(label)
        outputs = model(sample, labels=labels)
        loss, logits = outputs

        pred = torch.argmax(F.softmax(logits), dim=1)
        correct = pred.eq(labels)
        total_correct += correct.sum().item()
        total_len += len(labels)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

        if itr % p_itr == 0:
            !date "+[%F %R:%S] Training..."
            print(f"[Epoch {epoch+1}/{epochs}] Iteration {itr} -> Train Loss: {total_loss/p_itr:.4f}, Accuracy: {total_correct/total_len:.4f}")
            total_loss = 0
            total_len = 0
            total_correct = 0

        itr+=1

In [ ]:
!date "+[%F %R:%S] [DONE] Model Fine-tuning"
print(f"td={time.time() - t0:.3f}")
t0 = time.time()

In [ ]:
# evaluation
model.eval()

nsmc_eval_dataset = NsmcDataset(test_df)
eval_loader = DataLoader(nsmc_eval_dataset, batch_size=2, shuffle=False, num_workers=2)

total_loss = 0
total_len = 0
total_correct = 0

for text, label in eval_loader:
    encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]
    padded_list =  [e + [0] * (512-len(e)) for e in encoded_list]
    sample = torch.tensor(padded_list)
    sample, label = sample.to(device), label.to(device)
    labels = torch.tensor(label)
    outputs = model(sample, labels=labels)
    _, logits = outputs

    pred = torch.argmax(F.softmax(logits), dim=1)
    correct = pred.eq(labels)
    total_correct += correct.sum().item()
    total_len += len(labels)

print(f"Test accuracy: {total_correct / total_len:.4f}")

In [ ]:
!date "+[%F %R:%S] [EXIT] HelloNSMC (using $CONDA_PREFIX)"
print(f"td={time.time() - t0:.3f}")